# Step 1: Import and Clean Dataset from Sources
We import retraction data that we collected via queries from Crossref, Scopus, and WOS, and received directly from Retraction Watch. The code cleans and deduplicates the records. Then the final output of this code notebook is one merged file called the "Known Retraction List". 

In [ ]:
!pip install crossrefapi
!pip install --upgrade xlrd
!pip install openpyxl
!pip install dataframe-image

In [ ]:
# set up Crossref API

from crossref.restful import Works, Etiquette
my_etiquette = Etiquette('My Project Name', 'My Project version', 'My Project URL', 'My contact email')
print(str(my_etiquette))

works = Works(etiquette=my_etiquette)

In [ ]:
import pandas as pd
import dataframe_image as dfi

from datetime import date
from timeit import default_timer as timer

In [ ]:
today = str(date.today())

In [ ]:
# Set path --- Link to the box folder with your name
# Download Box Desktop to copy the pathname

# Input
# Folder name: step1-inputfile
box_path_1 = {enterdirectorytofolder}

# Output
# Folder name: step1-outputfile
box_path_2 = {enterdirectorytofolder}

## 1. Merge multiple downloaded files from the same source (Optional)

In [ ]:
# Input Files: All downloaded files from the same source  

# Scopus
file_list = []

for file_num in range(1,3):
    path = box_path_1 + "scopus (" + str(file_num) + ").csv"
    file_list.append(path)

    
sp = pd.DataFrame()

for file_name in file_list:
    sp = sp.append(pd.read_csv(file_name), ignore_index=True)

    
# Output File: One merged CSV file for a source 
sp.to_csv(box_path_1 + today +'-scopus.csv')

In [ ]:
# Web of Science

# specify the path
file_list = []

for file_num in range(9,26):
    path = box_path_1 + 'WOS/savedrecs(' + str(file_num) + ').xls'
    file_list.append(path)

wos = pd.DataFrame()

for file_name in file_list:
    wos = wos.append(pd.read_excel(file_name), ignore_index=True)

wos['Database'] = 'Web of Science'

# Output File: One merged CSV file for a source 
wos.to_csv(box_path_1 + today + '-webofscience.csv')

## 2. Collect retraction data from Crossref

- Get the count of each document type and identify which categories are related to retracted publication
- Get the count of document type that retracted publication
- Collect data using crossrefapi
- Clean dataframe and export into csv

In [ ]:
# Check document type
# Find all retraction indexing wordings

work = Works()

update_type = works.facet('update-type')
update_type_df = pd.DataFrame(update_type['update-type']['values'], index=['count']).transpose().reset_index().rename(columns={'index':'document_type'})

update_type_df

In [ ]:
# get the count of retraction document type

update_type_retraction = update_type_df.loc[update_type_df['document_type'].isin(['retraction','Retraction', 'retracion', 'retration', 'withdrawal','removal', 'partial_retraction'])].set_index(['document_type'])


retracted_tags = update_type_retraction.index.to_list()

update_type_retraction.loc['Total Count'] = update_type_retraction['count'].sum()
update_type_retraction

In [ ]:
# Set up function to collect data from crossrefapi

def getmetadata(x):
        
    """
    This function fetches publication data from the Crossref API based on document type.
    Data includes DOI, publicatio date, author, titles, and URL of the publication.
    
    :param x: specify the document type e.g. 'retraction', 'withdrawal'
    :return: dataframe of specific records
    """
    
    start = timer() # set timer

    metadata = {'DOI':[],'issued':[],'URL':[],'title':[],'author':[],'container-title':[]}
    
    for i in work.filter(update_type = x).select('DOI','issued','URL','title','author','container-title'):
        metadata['DOI'].append(i['DOI'])
        metadata['issued'].append(i['issued'])
        metadata['URL'].append(i['URL'])      
        try:
            metadata['author'].append(i['author'])
            
        except:
            metadata['author'].append('null')
            
        metadata['title'].append(i['title'])
        metadata['container-title'].append(i['container-title'])
     
    end = timer()
    print(end - start)

    
    return pd.DataFrame(metadata)

In [ ]:
# Collect metadate from Crossref

crossref_df= pd.DataFrame()

for x in update_type_retraction:
    crossref_df = crossref_df.append(getmetadata(x), ignore_index=True)

In [ ]:
# Format the dataframe

crossref_df = crossref_df.rename(columns={'issued':'Year', 'container-title': 'Journal','title': 'Title', 'author': 'Author'})

year=[]
title=[]
journal=[]
author=[]

for i in range(len(crossref_df)):
    year.append(crossref_df['Year'][i]['date-parts'][0][0])
    title.append(crossref_df['Title'][i][0])
    journal.append(crossref_df['Journal'][i][0])
    author.append(crossref_df['Author'][i])

    
crossref_df['Year'] = year
crossref_df['Title'] = title
crossref_df['Journal'] = journal
crossref_df['Author'] = author

crossref_df.Year.unique()

crossref_df.fillna(0)
crossref_df.replace('null','')


# Output File: One CSV files listing retracted publication
crossref_df.to_csv(box_path_1 + today + '-crossref.csv')

In [ ]:
# Get additional data from Crossref -- Category

url_base = 'https://api.crossref.org/works/{}'

crossref_cat = crossref_retracted['DOI'].tolist()
crossref_cat_chunks = np.array_split(crossref_cat, 147)


output = box_path_2 + 'crossrefcategory_2.tsv'
with open(output, 'a') as fout:
    line = ['DOI', 'Category']
    line = '\t'.join(line) + '\n'
    fout.write(line)
    
for chunk in crossref_cat_chunks:
    start_time = time.time()
    for doi in chunk:
        try:
            url = url_base.format(doi)
            result = requests.get(url)
            result = result.json()

            cat = result['message']['subject']

            new_line = [doi, str(cat)]
            new_line = '\t'.join(new_line) + '\n'
            with open(output, 'a') as fout:
                fout.write(new_line)
        except:
            print(doi)

        end_time = time.time()

    process_time = end_time - start_time
    
    time.sleep(3)

print(process_time)

In [ ]:
crossrefcat = pd.read_table(box_path_2 + 'crossrefcategory_2.tsv').fillna('')
crossrefcat['Category_str'] = [''.join(map(str, l)) for l in crossrefcat['Category']]

crossrefcat['Category_str'] = crossrefcat.Category_str.str.replace('[','').str.replace(']','')
crossrefcat.Category_str.value_counts()[:10]

## 3. Clean and deduplicate retraction data of each source 

In [ ]:
# Imput Files: Four files (one from each source) listing retracted publication.


# Crossref
crossref = pd.read_csv(box_path_1 + '2023-04-05-crossref.csv')
crossref['source'] = 'Crossref'

print(crossref.shape)
print(crossref.info())
print(crossref.head())
               
               
# Retraction Watch
retractionwatch = pd.read_excel(box_path_1 + '2023-03-27-RetractionWatch.xlsx', engine='openpyxl').rename(
    columns={'OriginalPaperDOI':'DOI', 
             'OriginalPaperPubMedID': 'PubMedID', 
             'OriginalPaperDate': 'Year'})

retractionwatch['source'] = 'Retraction Watch'

retractionwatch['PubMedID'] = pd.to_numeric(retractionwatch['PubMedID']).fillna(0).astype(str)
retractionwatch["Year"] = pd.to_datetime(retractionwatch["Year"]).dt.strftime("%Y").fillna(0).astype(int)
retractionwatch['RetractionYear'] = pd.to_datetime(date['RetractionDate']).dt.strftime("%Y").fillna(0).astype(int)
retractionwatch['DOI'] = retractionwatch['DOI'].fillna(0)

print(retractionwatch.shape)
print(retractionwatch.info())
print(retractionwatch.head())


# Scopus
scopus = pd.read_csv(box_path_1 + '2023-04-05-scopus.csv').rename(
    columns={'Authors':'Author',
             'Source':'source',
             'Titles':'Title',
             'Source title':'Journal',
             'PubMed ID': 'PubMedID'})

scopus['Year'] = pd.to_numeric(scopus['Year']).fillna(0).astype(int)
scopus['PubMedID'] = pd.to_numeric(scopus['PubMedID']).fillna(0).astype(str)


print(scopus.shape)
print(scopus.info())
print(scopus.head())

               
# Web of Science
wos = pd.read_csv(box_path_1 + '2023-04-05-webofscience.csv').rename(
    columns={'Authors':'Author', 
             'Article Title': 'Title', 
             'Source Title': 'Journal', 
             'Publication Year': 'Year', 
             'Pubmed Id': 'PubMedID'})

wos['PubMedID'] = pd.to_numeric(wos['PubMedID']).fillna(0).astype(str)
wos['source'] = 'Web of Science'

print(wos.shape)
print(wos.info())
print(wos.head())

In [ ]:
# Create function to clean and deduplicate retraction data of each source

def checkindividualdataset(x):
    
    # Input dataframe names as x
    
    '''
    Clean and deplicate records based on DOIs.
    After removing duplicates, we will return the count and the list of records with DOI, 
    those without DOIs and duplicated records that will be dropped.
    '''
    
    # Step 1: We identify the unique records of each dataset based on DOI.
    # 'records_withDOI_hasdup': Identify records that have a valid DOI which should start with '10.'
    # 'records_withDOI': Drop the duplicates from the previous line.
    records_withDOI_hasdup= x.loc[x['DOI'].str.startswith('10.', na=False)]
    records_withDOI = records_withDOI_hasdup.drop_duplicates(subset=['DOI'], keep='first')

    
    # Step 2: We create two duplicate lists.
    # 'duplicated_records_all': Identify ALL duplicated records for reference and download for checking manually.
    # 'duplicated_records': Identify duplicated records to drop but keep only the first occurrence of each group of duplicates.
    duplicate_records_all = records_withDOI_hasdup.loc[records_withDOI_hasdup.duplicated(subset=['DOI'],keep=False), :]
    duplicate_records = records_withDOI_hasdup.loc[records_withDOI_hasdup.duplicated(subset=['DOI'],keep='first'), :]

    
    # Step 3: We get the count of records without DOI. Duplicates may exist since we could not use DOI to identify duplicate records
    records_withoutDOI = x.loc[~x['DOI'].str.startswith('10.', na=False)]    

    
    try:
        
        if len(x) == len(records_withDOI)+len(records_withoutDOI)+len(duplicate_records):
            return[len(x), len(records_withDOI),len(records_withoutDOI),len(duplicate_records), records_withDOI, records_withoutDOI, duplicate_records, duplicate_records_all]
            # return the count and items of each group
    
    except: 
        return('ERROR')

In [ ]:
# Run function checkindividualdataset(x) 
# Create two lists to store the count and items

dbtable = [] # A nested list which stores the records of each group in each source
ovtable = [] # Store the count of each group from each source and create a table for viewing
dblist = [scopus, wos, retractionwatch, crossref]

for x in dblist:
    dbtable.append(checkindividualdataset(x))

for x in range(0,len(dbtable)):
    ovtable.append(dbtable[x][0:4])


# Create a table showing the count of each group
overview = pd.DataFrame(ovtable)
overview.columns =['total', 'records_withDOI', 'records_withoutDOI', 'duplicate_records']
overview['source']= ['Scopus','Web of Science', 'Retraction Watch', 'Crossref']
overview.loc[len(overview)] = [overview.total.sum(), overview.records_withDOI.sum(), overview.records_withoutDOI.sum(), overview.duplicate_records.sum(),''] 

overview.insert(0, 'source', overview.pop('source'))
overview['source'][4]= 'Total'

print(overview)


## export table to png
dfi.export(overview, 'source_overview.png')

In [ ]:
# Take the nested list 'dblist' and export each group in each source individually.

def exportthreedatasets(x):
    
    if x == 'scopus':
        dbtable[0][4].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-recordswithdoi-' + x + '.csv')
        dbtable[0][5].to_csv(box_path_2 + today + '-recordsnodoi-' + x + '.csv')
        dbtable[0][7].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-duplicatedrecords-' + x + '.csv')

    if x == 'wos':
        dbtable[1][4].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-recordswithdoi-' + x + '.csv')
        dbtable[1][5].to_csv(box_path_2 + today + '-recordsnodoi-' + x + '.csv')
        dbtable[1][7].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-duplicatedrecords-' + x + '.csv')

    if x == 'retractionwatch':
        dbtable[2][4].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-recordswithdoi-' + x + '.csv')
        dbtable[2][5].to_csv(box_path_2 + today + '-recordsnodoi-' + x + '.csv')
        dbtable[2][7].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-duplicatedrecords-' + x + '.csv')
    
    if x == 'crossref':
        dbtable[3][4].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-recordswithdoi-' + x + '.csv')
        dbtable[3][5].to_csv(box_path_2 + today + '-recordsnodoi-' + x + '.csv')
        dbtable[3][7].sort_values(by=['DOI'], ascending=False).to_csv(box_path_2 + today + '-duplicatedrecords-' + x + '.csv')


In [ ]:
# Output Files: Three CSV files of cleaned data for each source listing:
# the records with DOI
# the records with no DOI
# all the duplicate records

for x in ['wos', 'scopus', 'retractionwatch', 'crossref']:
    exportthreedatasets(x)

## 4. Create a known retraction list 

In [ ]:
# Input Files: Four CSV files (one for each source) of the records with DOI. (Produced in step #3 above) 

scopus_retracted = pd.read_csv(box_path_2 + '2023-04-09-recordswithdoi-scopus.csv')
wos_retracted = pd.read_csv(box_path_2 + '2023-04-09-recordswithdoi-wos.csv')
retractionwatch_retracted = pd.read_csv(box_path_2 + '2023-04-09-recordswithdoi-retractionwatch.csv')
crossref_retracted = pd.read_csv(box_path_2 + '2023-04-09-recordswithdoi-crossref.csv')

print(scopus_retracted[['DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID']].dtypes)
print(wos_retracted[['DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID']].dtypes)
print(retractionwatch_retracted[['DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID']].dtypes)
print(crossref_retracted[['DOI', 'Author','Title', 'Year', 'Journal', 'source']].dtypes)

In [ ]:
# Merged into one full list 'The Known Retraction List'

merged_withdoi = pd.concat([wos_retracted, scopus_retracted, retractionwatch_retracted, crossref_retracted])
merged_withdoi = merged_withdoi[['DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID']].sort_values(by='DOI')

# Check if the number of records are consistent before and after merging.
if len(merged_withdoi) == len(wos_retracted)+len(scopus_retracted)+len(retractionwatch_retracted)+len(crossref_retracted): 
    print('full record count:', len(merged_withdoi))

else:
    print('ERROR: Inconsistent Counts')

In [ ]:
# Format the dataframe

knownretractionlist = merged_withdoi.groupby('DOI').agg({'Author':'first', 
                              'Title': 'last',
                              'Year': 'first', 
                              'Journal': 'last',
                              'source':'; '.join, 
                              'PubMedID':'first'}).reset_index()

print(knownretractionlist.shape())
print(knownretractionlist.info())
knownretractionlist.head()

In [ ]:
# Output File: One merged CSV file as final output 
knownretractionlist.to_csv(box_path_2 + today + '-knownretractionlist-0.csv')

In [ ]:
# Create a table showing the number of matched records in each group based on DOI.

df = pd.merge(
        pd.merge(
            pd.merge(
            scopus_retracted, wos_retracted,how='inner', on='DOI')
            , retractionwatch_retracted, how='inner', on='DOI')
            , crossref_retracted, how='inner', on='DOI')

df1 = pd.merge(scopus_retracted, wos_retracted,how='inner', on='DOI')
df2 = pd.merge(scopus_retracted, retractionwatch_retracted,how='inner', on='DOI')
df3 = pd.merge(wos_retracted, retractionwatch_retracted,how='inner', on='DOI')
df4 = pd.merge(crossref_retracted, scopus_retracted,how='inner', on='DOI')
df5 = pd.merge(crossref_retracted, wos_retracted,how='inner', on='DOI')
df6 = pd.merge(crossref_retracted, retractionwatch_retracted,how='inner', on='DOI')


crm = [len(crossref_retracted), len(df6), len(df4), len(df5), '-']
rwm = ['-', len(retractionwatch_retracted),len(df2), len(df3), '-']
spm = ['-', '-', len(scopus_retracted), len(df1), '-']
wosm = ['-', '-', '-', len(wos_retracted), '-']

allm = ['-','-','-','-', len(df)]

mtable = [crm, rwm, spm, wosm, allm]
match = pd.DataFrame(mtable)
match.columns = ['Crossref','Retraction Watch', 'Scopus', 'Web of Science', 'All Matched']
match['source']= ['Crossref','Retraction Watch', 'Scopus','Web of Science', 'All Matched']

# shift column 'database' to first position
match.insert(0, 'source', match.pop('source'))
match = match.set_index(match.columns[0])

match

In [ ]:
# export table
dfi.export(match, box_path_1 + today + 'matchingmatrix_overview.png', table_conversion="matplotlib")

**Run Step1.5 Notebook and return to the steps below**

In [ ]:
# add field of study tp each item
journallist = pd.read_csv(box_path_1 + '2023-04-13-journalcategory-knownretractionlist.csv').drop(['Unnamed: 0'], axis=1)
journallist.head()

In [ ]:
knownretractionlist['journal'] = knownretractionlist['Journal'].str.lower()

knownretractionlist = knownretractionlist.merge(journallist, left_on='journal', right_on='JournalandConferenceProceedings', how='left').drop(columns=['journal','JournalandConferenceProceedings'], axis=1)

print(knownretractionlist.shape())
print(knownretractionlist.info())
knownretractionlist.head()

In [ ]:
# Output File: One merged CSV file as final output 

knownretractionlist.to_csv(box_path_2 + today + '-knownretractionlist-1.csv')